In [ ]:
# Packages
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.svm import SVC
from sklearn import tree
import graphviz
import matplotlib.pyplot as plt

# **Importation Analyse et Prétraitement des données**

In [ ]:
data_0 = pd.read_csv('/kaggle/input/multiple-disease-prediction/Blood_samples_dataset_balanced_2(f).csv')
data_1 = pd.read_csv('/kaggle/input/multiple-disease-prediction/blood_samples_dataset_test.csv')

In [ ]:
data = pd.concat([data_0, data_1])
print(data.shape)
pd.set_option("display.max_columns", None)
data.sample(10)

In [ ]:
print(data.info())
print(data.isnull().sum())

In [ ]:
print("Nbre lignes dupliquées: ", data_0.duplicated().sum())
data = data.drop_duplicates()
print(data.shape)

In [ ]:
# Informations statistiques
table = np.array(data.drop(columns=["Disease"]))
print(np.max(table))
print(np.min(table))

In [ ]:
print(data["Disease"].value_counts(normalize=True))
data["Disease"] = LabelEncoder().fit_transform(data["Disease"])
plt.figure(figsize=(20, 15))
sn.heatmap(data.corr(), annot=True)

# **Modélisation**

In [ ]:
x = data.drop(columns=["Disease"])
y = data["Disease"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.30)
model_1 = LogisticRegression()
model_2 = KNeighborsClassifier(n_neighbors=25)
model_3 = DecisionTreeClassifier()
model_4 = RandomForestClassifier(n_estimators=10)
model_5 = SVC()
for model in (model_1, model_2, model_3, model_4, model_5):
  model.fit(x_train, y_train)
  print("% deprécision pour ",model ,"est:", model.score(x_test, y_test)*100, "%")

Choix d'un modéle : **DésisionTreeClassifier**

In [ ]:
model = DecisionTreeClassifier(criterion="gini", max_depth=100)
model.fit(x_train, y_train)
prediction = model.predict(x_test)
print("% qualité de prédiction: ", metrics.accuracy_score(y_test, prediction)*100, "%")
print(metrics.classification_report(y_test, prediction))

In [ ]:
matrix_confusion = pd.crosstab(y_test, prediction, rownames=["Actual"], colnames=["Predicted"])
sn.heatmap(matrix_confusion, annot=True)

In [ ]:
data_ = tree.export_graphviz(model)
fig = graphviz.Source(data_)
fig